In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install "anomalib[vlm_clip]"
!pip install open-clip-torch


  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.9/44.9 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 241.8/241.8 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.0/846.0 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 983.2/983.2 kB 34.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 851.8/851.8 kB 38.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 64.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 75.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 760.5/760.5 kB 35.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.5/849.5 kB 43.9 MB/s eta 0:00:00
  Created wheel for freia: filename=FrEIA-0.2-py3-none-any.whl size=42763 sha256=492c4176c35dc91ccd67b9f20fb6aa7ab6753a8ecf2f928262eeda4d4d5bcb74
  Stored in dir

In [ ]:
from anomalib.data import MVTecAD
from anomalib.models import Ganomaly
from anomalib.engine import Engine

In [ ]:
import os, glob
from PIL import Image

base = "/content/drive/MyDrive/Colab Notebooks/data/battery_small"
folders = [
    f"{base}/train/good",
    f"{base}/test/good",
    f"{base}/test/overheat",
    f"{base}/test/thermal_imbalance",
    f"{base}/test/spatial_tape",
    f"{base}/ground_truth/overheat",
    f"{base}/ground_truth/thermal_imbalance",
    f"{base}/ground_truth/spatial_tape"
]

for folder in folders:
    if not os.path.isdir(folder):
        continue
    for p in glob.glob(os.path.join(folder, "*.jpg")) + glob.glob(os.path.join(folder, "*.JPG")):
        out = os.path.splitext(p)[0] + ".png"
        if not os.path.exists(out):
            Image.open(p).convert("RGB").save(out)
print("Done converting to PNG.")

Done converting to PNG.


In [ ]:
from anomalib.data import MVTecAD
from anomalib.models import Ganomaly
from anomalib.engine import Engine

# Initialize model and data
datamodule = MVTecAD(
    root="/content/drive/MyDrive/Colab Notebooks/data",
    category="battery_small",       # folder under your_dataset/
    num_workers=2
)
datamodule.setup()
print("Train samples:", len(datamodule.train_data.samples))
print("Test samples:", len(datamodule.test_data.samples))

Train samples: 337
Test samples: 371


In [ ]:
model = Ganomaly(
    batch_size=32,
    n_features=6,
    latent_vec_size=100,
    wadv=1,
    wcon=50,
    wenc=1,
)

In [ ]:
engine = Engine(enable_progress_bar=False)
engine.fit(model=model, datamodule=datamodule)

INFO:lightning_fabric.utilities.rank_zero:GPU available: False, used: False
INFO:lightning_fabric.utilities.rank_zero:TPU available: False, using: 0 TPU cores
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/loops/utilities.py:73: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.


┏━━━┳━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━┳━━━━━━━┓
┃   ┃ Name               ┃ Type              ┃ Params ┃ Mode  ┃ FLOPs ┃
┡━━━╇━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━╇━━━━━━━┩
│ 0 │ pre_processor      │ PreProcessor      │      0 │ train │     0 │
│ 1 │ post_processor     │ PostProcessor     │      0 │ train │     0 │
│ 2 │ evaluator          │ Evaluator         │      0 │ train │     0 │
│ 3 │ model              │ GanomalyModel     │  188 M │ train │     0 │
│ 4 │ generator_loss     │ GeneratorLoss     │      0 │ train │     0 │
│ 5 │ discriminator_loss │ DiscriminatorLoss │      0 │ train │     0 │
└───┴────────────────────┴───────────────────┴────────┴───────┴───────┘

Trainable params: 188 M                                                                                            
Non-trainable params: 0                                                                                            
Total params: 188 M                                                                                                
Total estimated model params size (MB): 754                                                                        
Modules in train mode: 112                                                                                         
Modules in eval mode: 0                                                                                            
Total FLOPs: 0

/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/module.py:522: You called `self.log('generator_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`
/usr/local/lib/python3.12/dist-packages/lightning/pytorch/core/module.py:522: You called `self.log('discriminator_loss', ..., logger=True)` but have no logger configured. You can enable one by doing `Trainer(logger=ALogger(...))`


In [ ]:
predictions = engine.predict(model=model, datamodule=datamodule)

In [ ]:
test_metrics = engine.test(model=model, datamodule=datamodule)
print(test_metrics)  # e.g. {'image_auroc': ..., 'pixel_auroc': ..., ...}
